In [2]:
import pandas as pd
import xml.etree.ElementTree as etree


tree = etree.parse("muni-fi-spr16.xml")
root = tree.getroot()

columns_problem = ["Name", "NrDays", "SlotsPerDay", "NrWeeks"]
columns_optimization = ["Time", "Room", "Distribution", "Student"]
columns_rooms = ["RoomId", "Capacity", "PreviousRoomIdList", "TravelTimeValueList", "UnavailableDaysList", "UnavailableStartList", "UnavailableLengthList", "UnavailableWeeksList"]
columns_courses = ["CourseId", "ConfigIdList"]
columns_configs = ["ConfigId", "SubpartIdList"]
columns_subparts = ["SubpartId", "ClassIdList"]
columns_classes = ["ClassId", "ClassLimit", "ClassParent"]
columns_distributions = ["DistributionId", "DistributionType", "DistributionRequired", "DistributionPenalty", "ClassesList"]
columns_students = ["StudentId", "CoursesList"]
columns_relation_class_room = ["ClassId", "RoomId", "RoomPenalty"]
columns_relation_class_time = ["ClassId", "TimeDays", "TimeStart", "TimeLength", "TimeWeeks", "TimePenalty"]

df_problem = pd.DataFrame(columns = columns_problem)
df_optimization = pd.DataFrame(columns = columns_optimization)
df_rooms = pd.DataFrame(columns = columns_rooms)
df_courses = pd.DataFrame(columns = columns_courses)
df_configs = pd.DataFrame(columns = columns_configs)
df_subparts = pd.DataFrame(columns = columns_subparts)
df_classes = pd.DataFrame(columns = columns_classes)
df_distributions = pd.DataFrame(columns = columns_distributions)
df_students = pd.DataFrame(columns = columns_students)
df_relation_class_room = pd.DataFrame(columns = columns_relation_class_room)
df_relation_class_time = pd.DataFrame(columns = columns_relation_class_time)




df_problem = df_problem.append({"Name": root.attrib.get("name"), "NrDays": root.attrib.get("nrDays"), "SlotsPerDay": root.attrib.get("slotsPerDay"), "NrWeeks": root.attrib.get("nrWeeks")}, ignore_index = True)

for node in root:
    if node.tag == "optimization":
        df_optimization = df_optimization.append({"Time": node.attrib.get("time"), "Room": node.attrib.get("room"), "Distribution": node.attrib.get("distribution"), "Student": node.attrib.get("student")}, ignore_index = True)
    
    elif node.tag == "rooms":
        for room in node:
            roomid = room.attrib.get("id")
            cap = room.attrib.get("capacity")
            prevroomlist = []
            valuelist = []
            dayslist = []
            startlist = []
            lengthlist = []
            weekslist = []
            if len(list(child.tag for child in room.iter() if child is not room)) > 0:
                for child in room:
                    if child.tag == "travel":
                        prevroomid = child.attrib.get("room")
                        value = child.attrib.get("value")
                        prevroomlist.append(prevroomid)
                        valuelist.append(value)
                    elif child.tag == "unavailable":
                        days = child.attrib.get("days")
                        start = child.attrib.get("start")
                        length = child.attrib.get("length")
                        weeks = child.attrib.get("weeks")
                        dayslist.append(days)
                        startlist.append(start)
                        lengthlist.append(length)
                        weekslist.append(weeks)
            if len(prevroomlist) == 0:
                prevroomlist = None
            if len(valuelist) == 0:
                valuelist = None
            if len(dayslist) == 0:
                dayslist = None
            if len(startlist) == 0:
                startlist = None
            if len(lengthlist) == 0:
                lengthlist = None
            if len(weekslist) == 0:
                weekslist = None
            df_rooms = df_rooms.append(pd.Series([roomid, cap, prevroomlist, valuelist, dayslist, startlist, lengthlist, weekslist], index = columns_rooms), ignore_index = True)
            
    elif node.tag == "courses":
        for course in node:
            #print(course.tag)
            courseid = course.attrib.get("id")
            configlist = []
            for config in course:
                #print(config.tag)
                configid = config.attrib.get("id")
                configlist.append(configid)
                subpartlist = []
                for subpart in config:
                    #print(subpart.tag)
                    subpartid = subpart.attrib.get("id")
                    subpartlist.append(subpartid)
                    claselist = []
                    for clase in subpart:
                        #print(clase.tag)
                        claseid = clase.attrib.get("id")
                        claselist.append(claseid)
                        claselimit = clase.attrib.get("limit")
                        claseparent = clase.attrib.get("parent") if clase.attrib.get("parent") is not None else None
                        df_classes = df_classes.append(pd.Series([claseid, claselimit, claseparent], index = columns_classes), ignore_index = True)
                        if len(list(child.tag for child in clase.iter() if child is not clase)) > 0:
                            for child in clase:
                                #print(child.tag)
                                if child.tag == "room":
                                    roomid = child.attrib.get("id")
                                    roompenalty = child.attrib.get("penalty")
                                    df_relation_class_room = df_relation_class_room.append(pd.Series([claseid, roomid, roompenalty], index = columns_relation_class_room), ignore_index = True)
                                elif child.tag == "time":
                                    timedays = child.attrib.get("days")
                                    timestart = child.attrib.get("start")
                                    timelength = child.attrib.get("length")
                                    timeweeks = child.attrib.get("weeks")
                                    timepenalty = child.attrib.get("penalty")
                                    df_relation_class_time = df_relation_class_time.append(pd.Series([claseid, timedays, timestart, timelength, timeweeks, timepenalty], index = columns_relation_class_time), ignore_index = True)
                    df_subparts = df_subparts.append(pd.Series([subpartid, claselist], index = columns_subparts), ignore_index = True)
                df_configs = df_configs.append(pd.Series([configid, subpartlist], index = columns_configs), ignore_index = True)
            df_courses = df_courses.append(pd.Series([courseid, configlist], index = columns_courses), ignore_index = True)
                                    
    elif node.tag == "distributions":
        contador = 1
        for distribution in node:
            #print(distribution.tag)
            distribid = contador
            distribtype = distribution.attrib.get("type")
            distribrequired = distribution.attrib.get("required") if distribution.attrib.get("required") is not None else None
            distribpenalty = distribution.attrib.get("penalty") if distribution.attrib.get("penalty") is not None else None
            listaclases = []
            for clase in distribution:
                #print(clase.tag)
                listaclases.append(clase.attrib.get("id"))
            df_distributions = df_distributions.append(pd.Series([distribid, distribtype, distribrequired, distribpenalty, listaclases], index = columns_distributions), ignore_index = True)
            contador += 1
             
    elif node.tag == "students":
        for student in node:
            studentid = student.attrib.get("id")
            listacourses = []
            for course in student:
                listacourses.append(course.attrib.get("id"))
            df_students = df_students.append(pd.Series([studentid, listacourses], index = columns_students), ignore_index = True)
            
            


print("################################################################################################################")
print("DataFrame Problem")
display(df_problem)
print("################################################################################################################")
print("DataFrame Optimization")
display(df_optimization)
print("################################################################################################################")
print("DataFrame Rooms")
display(df_rooms)
print("################################################################################################################")
print("DataFrame Courses")
display(df_courses)
print("################################################################################################################")
print("DataFrame Configs")
display(df_configs)
print("################################################################################################################")
print("DataFrame Subparts")
display(df_subparts)
print("################################################################################################################")
print("DataFrame Classes")
display(df_classes)
print("################################################################################################################")
print("DataFrame Distributions")
display(df_distributions)
print("################################################################################################################")
print("DataFrame Students")
display(df_students)
print("################################################################################################################")
print("DataFrame RelationClassRoom")
display(df_relation_class_room)
print("################################################################################################################")
print("DataFrame RelationClassTime")
display(df_relation_class_time)
print("################################################################################################################")

################################################################################################################
DataFrame Problem


,Name,NrDays,SlotsPerDay,NrWeeks
0,muni-fi-spr16,7,288,15


################################################################################################################
DataFrame Optimization


,Time,Room,Distribution,Student
0,3,1,10,5


################################################################################################################
DataFrame Rooms


...

################################################################################################################
DataFrame RelationClassRoom


,ClassId,RoomId,RoomPenalty
0,1,18,0
1,15,23,0
2,15,8,0
3,15,12,0
4,16,23,4
...,...,...,...
2404,526,27,4
2405,526,30,0
2406,526,6,0
2407,526,7,1


################################################################################################################
DataFrame RelationClassTime


,ClassId,TimeDays,TimeStart,TimeLength,TimeWeeks,TimePenalty
0,1,0000100,168,46,011110111111110,0
1,2,1000000,96,46,011111011111110,0
2,2,0000100,96,46,011110111111110,0
3,2,1000000,108,46,011111011111110,0
4,2,0000100,108,46,011110111111110,0
...,...,...,...,...,...,...
9551,571,1000000,181,18,010101010101010,0
9552,572,1000000,200,18,011111011111110,0
9553,573,0001000,96,18,010101010101010,0
9554,574,0001000,115,18,011111111111110,0


################################################################################################################


In [61]:
#Pasar a diccionarios las tablas de pandas 
dicc_cursos = {}
for index, row in df_courses.iterrows():
    dicc_cursos[int(row["CourseId"])] = [int(x) for x in row["ConfigIdList"]]   
    
dicc_configuraciones = {}
for index, row in df_configs.iterrows():
    dicc_configuraciones[int(row["ConfigId"])] = [int(x) for x in row["SubpartIdList"]]

dicc_subpartes = {}
for index, row in df_subparts.iterrows():
    dicc_subpartes[int(row["SubpartId"])] = [int(x) for x in row["ClassIdList"]]

dicc_clases = {}
for index, row in df_classes.iterrows():
    dicc_clases[int(row["ClassId"])] = [int(x) for x in row["ClassLimit"]]


In [12]:
#CONJUNTOS

In [62]:
#Asignaciones de cursos hechas a mano porque tienen más de una configuración
cursos_sin_asignar = {}
configuraciones_sin_asignar = {}
subpartes_sin_asignar = {}

#Conjunto de cursos
cursos = {}
for curso in dicc_cursos:
    if len(dicc_cursos[curso]) == 1:
        cursos[curso] = curso
    else:
        cursos_sin_asignar[curso] = dicc_cursos[curso]

#Conjunto de configuraciones
configuraciones = {}
for configuracion in dicc_configuraciones:
    for configs in cursos_sin_asignar.values():
        if configuracion not in configs:
            configuraciones[configuracion] = configuracion
        else:
            configuraciones_sin_asignar[configuracion] = dicc_configuraciones[configuracion]

#Conjunto de Subpartes
subpartes = {}
for subparte in dicc_subpartes:
    for subparts in configuraciones_sin_asignar.values():
        if subparte not in subparts:
            subpartes[subparte] = subparte
        else:
            subpartes_sin_asignar[subparte] = dicc_subpartes[subparte]

#Conjunto Clases
clases = {}
for clase in dicc_clases:
    for classes in subpartes_sin_asignar.values():
        if clase not in classes:
            clases[clase] = clase

#D distribuciones
distribuciones = {}
for index, row in df_distributions.iterrows():
    if row["DistributionType"] == "SameAttendees" or row["DistributionType"] == "SameTime" or row["DistributionType"] == "SameRoom" or row["DistributionType"] == "SameDays" or row["DistributionType"] == "Precedence" or row["DistributionType"] == "NotOverlap" or row["DistributionType"] == "SameWeeks":
        distribuciones[row["DistributionId"]] = row["DistributionId"]

In [ ]:
#SUBCONJUNTOS

In [139]:
#Tc = periodos disponibles para la clase c, donde t=(Wt, Dt, ht, lent) 
dicc_Tc = {}
dicc_tiempos = {}
for index,row in df_relation_class_time.iterrows():
    if int(row["ClassId"]) not in dicc_Tc.keys():
        dicc_Tc[int(row["ClassId"])] = []
    dicc_Tc[int(row["ClassId"])].append([row["TimeWeeks"], row["TimeDays"], int(row["TimeStart"]), int(row["TimeLength"])])
    dicc_tiempos[index] = [row["TimeWeeks"], row["TimeDays"], int(row["TimeStart"]), int(row["TimeLength"])]
#print(dicc_Tc)

SyntaxError: unexpected EOF while parsing (<ipython-input-139-a1c899b5265b>, line 9)

In [68]:
#Sc = salas disponibles para la clase c
dicc_Sc = {}
for index, row in df_relation_class_room.iterrows():
    if int(row["ClassId"]) not in dicc_Sc.keys():
        dicc_Sc[int(row["ClassId"])] = []
    dicc_Sc[int(row["ClassId"])].append(int(row["RoomId"]))
#print(dicc_Sc)

In [66]:
#Fk = configuraciones del curso k
dicc_Fk = {}
for curso in dicc_cursos:
    if len(dicc_cursos[curso]) == 1:
        dicc_Fk[curso] = dicc_cursos[curso]
#print(dicc_Fk)

In [71]:
#Pf = subpartes de la configuracion f
dicc_Pf = {}
for configuracion in dicc_configuraciones:
    dicc_Pf[configuracion] = dicc_configuraciones[configuracion]
#print(dicc_Pf)

In [74]:
#Cp = clases de la subparte p
dicc_Cp = {}
for subparte in dicc_subpartes:
    dicc_Cp[subparte] = dicc_subparts[subparte]
#print(dicc_Cp)

In [84]:
#UTs = periodo no disponible para la sala s
dicc_UTs = {}
for index, col in df_rooms.iterrows():
    if col["UnavailableDaysList"] != None:
        sem_no_disp = col["UnavailableWeeksList"]
        dias_no_disp = col["UnavailableDaysList"]
        hor_no_disp = [int(x) for x in col["UnavailableStartList"]]
        largo_no_disp = [int(x) for x in col["UnavailableLengthList"]]
        dicc_UTs[int(col["RoomId"])] = [sem_no_disp, dias_no_disp, hor_no_disp, largo_no_disp]
    else:
        dicc_UTs[int(col["RoomId"])] = [] 
#print(dicc_UTs)

In [ ]:
#SUBCONJUNTOS DE DISTRIBUCIONES

In [86]:
## D1 = distribuciones de tipo 1 (SameAttendees) lista con id's
#distribución SameAttendees dura
D1 =[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameAttendees" and row["DistributionRequired"]=="true":
        D1.append(int(row["DistributionId"]))
#print(D1)

#distribución SameAttendees blanda
D1_B =[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameAttendees" and row["DistributionRequired"]==None:
        D1_B.append(int(row["DistributionId"]))
#print(D1_B)

In [87]:
## D2 = distribuciones de tipo 2 (SameStart) lista con id's
#distribución SameStart dura
D2 =[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameStart" and row["DistributionRequired"]=="true":
        D2.append(int(row["DistributionId"]))
#print(D2)

#distribución SameStart blanda
D2_B =[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameStart" and row["DistributionRequired"]==None:
        D2_B.append(int(row["DistributionId"]))
#print(D2_B)

In [88]:
## D3 = D3=distribuciones de tipo 3 (SameRoom) lista con id's
#distribución SameRoom dura
D3=[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameRoom" and row["DistributionRequired"]=="true":
        D3.append(int(row["DistributionId"]))
#print(D3)

#distribución SameRoom blanda
D3_B =[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameRoom" and row["DistributionRequired"]==None:
        D3_B.append(int(row["DistributionId"]))
#print(D3_B)

In [90]:
## D4 = distribuciones de tipo 4 (SameTime) lista con id's
#distribución SameTime dura
D4=[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameTime" and row["DistributionRequired"]=="true":
        D4.append(int(row["DistributionId"]))
#print(D4)

#distribución SameTime blanda
D4_B =[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameTime" and row["DistributionRequired"]==None:
        D4_B.append(int(row["DistributionId"]))
#print(D4_B)

In [92]:
## D5 = distribuciones de tipo 5 (SameDays) lista con id's
#distribución SameDays dura
D5=[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameDays" and row["DistributionRequired"]=="true":
        D5.append(int(row["DistributionId"]))
#print(D5)

#distribución SameDays blanda
D5_B =[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameDays" and row["DistributionRequired"]==None:
        D5_B.append(int(row["DistributionId"]))
#print(D5_B)

In [93]:
## D6 = distribuciones de tipo 6 (SameWeeks) lista con id's
#distribución SameWeeks dura
D6=[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameWeeks" and row["DistributionRequired"]=="true":
        D6.append(int(row["DistributionId"]))
#print(D6)

#distribución SameWeeks blanda
D6_B =[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameWeeks" and row["DistributionRequired"]==None:
        D6_B.append(int(row["DistributionId"]))
#print(D6_B)

In [94]:
## D7 = distribuciones de tipo 7 (Precendence) lista con id's
#distribución Precendence dura
D7=[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="Precedence" and row["DistributionRequired"]=="true":
        D7.append(int(row["DistributionId"]))
#print(D7)

#distribución Precendence blanda
D7_B =[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="Precedence" and row["DistributionRequired"]==None:
        D7_B.append(int(row["DistributionId"]))
#print(D7_B)

In [95]:
## D8 = distribuciones de tipo 8 (NotOverlap) lista con id's
#distribución NotOverlap dura
D8=[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="NotOverlap" and row["DistributionRequired"]=="true":
        D8.append(int(row["DistributionId"]))
#print(D8)

#distribución NotOverlap blanda
D8_B =[]
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="NotOverlap" and row["DistributionRequired"]==None:
        D8_B.append(int(row["DistributionId"]))
#print(D8_B)

In [97]:
#SAi=clases de la distribución SameAttendees i (dura)
Sa = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameAttendees" and row["DistributionRequired"]=="true":
        Sa[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            Sa[int(row["DistributionId"])].append(int(i))
#print(Sa)

#SABi=clases de la distribución SameAttendees i (blanda)
SaB = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameAttendees" and row["DistributionRequired"]==None:
        SaB[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            SaB[int(row["DistributionId"])].append(int(i))
#print(SaB)

In [98]:
#SSi =clases de la distribución SameStart i (dura)
SS = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameStart" and row["DistributionRequired"]=="true":
        SS[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            SS[int(row["DistributionId"])].append(int(i))
#print(SS)

#SSBi=clases de la distribución SameStart i  (blanda)
SSB = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameStart" and row["DistributionRequired"]==None:
        SSB[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            SSB[int(row["DistributionId"])].append(int(i))
#print(SSB)

In [99]:
#SRi =clases de la distribución SameRoom i (dura) (dura)
SR = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameRoom" and row["DistributionRequired"]=="true":
        SR[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            SR[int(row["DistributionId"])].append(int(i))
#print(SR)

#SRBi =clases de la distribución SameRoom i (blanda)
SRB = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameRoom" and row["DistributionRequired"]==None:
        SRB[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            SRB[int(row["DistributionId"])].append(int(i))
#print(SRB)

In [100]:
#STi=clases de la distribución SameTime i (dura)
ST = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameTime" and row["DistributionRequired"]=="true":
        ST[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            ST[int(row["DistributionId"])].append(int(i))
#print(ST)

#STi=clases de la distribución SameTime i  (blanda)
STB = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameTime" and row["DistributionRequired"]==None:
        STB[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            STB[int(row["DistributionId"])].append(int(i))
#print(STB)

In [101]:
#SDi=clases de la distribución SameDays i (dura)
SD = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameDays" and row["DistributionRequired"]=="true":
        SD[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            SD[int(row["DistributionId"])].append(int(i))
#print(SD)

#SDBi=clases de la distribución SameDays i (blanda)
SDB = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameDays" and row["DistributionRequired"]==None:
        SDB[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            SDB[int(row["DistributionId"])].append(int(i))
#print(SDB)

In [102]:
#PRi=clases de la distribución Precedence i (dura)
PR = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="Precedence" and row["DistributionRequired"]=="true":
        PR[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            PR[int(row["DistributionId"])].append(int(i))
#print(PR)

#PRBi=clases de la distribución Precedence i (blanda)
PRB = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="Precedence" and row["DistributionRequired"]==None:
        PRB[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            PRB[int(row["DistributionId"])].append(int(i))
#print(PRB)

In [103]:
#NOi=clases de la distribución NotOverlap i (dura)
NO = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="NotOverlap" and row["DistributionRequired"]=="true":
        NO[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            NO[int(row["DistributionId"])].append(int(i))
#print(NO)

#NOBi=clases de la distribución NotOverlap i (blanda)
NOB = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="NotOverlap" and row["DistributionRequired"]==None:
        NOB[int(row["DistributionId"])] = []
        for i in row["ClassesList"]:
            NOB[int(row["DistributionId"])].append(int(i))
#print(NOB)

In [ ]:
#PARÁMETROS

In [130]:
dicc_T = {}
for index,row in df_relation_class_time.iterrows():
    if int(row["ClassId"]) not in dicc_T.keys():
        dicc_T[int(row["ClassId"])] = []
    dicc_T[int(row["ClassId"])].append([row["TimeWeeks"], row["TimeDays"], int(row["TimeStart"]), int(row["TimeLength"])])
#print(dicc_Tc)

In [131]:
#t[semanas]=semanas correspondientes al periodo t Wt
dicc_Wt = {}
for tiempo in dicc_T:
    for t in dicc_T[tiempo]:
        if tiempo not in dicc_Wt.keys():
            dicc_Wt[tiempo] = []
        dicc_Wt[tiempo].append(t[0])
#print(dicc_Wt)

In [133]:
#t[dias]=días correspondientes al periodo t Dt
dicc_Dt = {}
for tiempo in dicc_T:
    for t in dicc_T[tiempo]:
        if tiempo not in dicc_Dt.keys():
            dicc_Dt[tiempo] = []
        dicc_Dt[tiempo].append(t[1])
#print(dicc_Dt)

In [134]:
#t[slot}=slot en el que inicia el periodo t Ht
dicc_Ht = {}
for tiempo in dicc_T:
    for t in dicc_T[tiempo]:
        if tiempo not in dicc_Ht.keys():
            dicc_Ht[tiempo] = []
        dicc_Ht[tiempo].append(t[2])
#print(dicc_Ht)

In [135]:
#t[len]=duración del periodo t Lh
dicc_Lt = {}
for tiempo in dicc_T:
    for t in dicc_T[tiempo]:
        if tiempo not in dicc_Lt.keys():
            dicc_Lt[tiempo] = []
        dicc_Lt[tiempo].append(t[3])
#print(dicc_Lt)

In [128]:
#PTc,t=penalización asociada a asignar la clase c en el tiempo t
dicc_PTct = {}
for index, row in df_relation_class_time.iterrows():
    llave = (int(row["ClassId"]),int(index))
    dicc_PTct[llave] = int(row["TimePenalty"]) 
#print(dicc_PTct)

In [127]:
#PSc,s=penalización asociada a asignar la clase c en la sala s
dicc_PScs = {}
for index, row in df_relation_class_room.iterrows():
    llave = (int(row["ClassId"]),int(row["RoomId"]))
    dicc_PScs[llave] = int(row["RoomPenalty"]) 
#print(dicc_PScs)

In [3]:
#PD1,i=penalización asociada a la distribución i de tipo 1 (SamAttendees)
dicc_PD1i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "SameAttendees":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD1i[int(row["DistributionId"])] = dp

#PD2,i=penalización asociada a la distribución i de tipo 2 (SameStart)
dicc_PD2i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "SameStart":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD2i[int(row["DistributionId"])] = dp
        
#PD3,i=penalización asociada a la distribución i de tipo 3 (SameRoom)
dicc_PD3i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "SameRoom":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD3i[int(row["DistributionId"])] = dp
        
#PD4,i=penalización asociada a la distribución i de tipo 4(SameTime)       
dicc_PD4i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "SameTime":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD4i[int(row["DistributionId"])] = dp

#PD5,i=penalización asociada a la distribución i de tipo 5 (SameDays) 
dicc_PD5i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "SameDays":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD5i[int(row["DistributionId"])] = dp
        
#PD6,i=penalización asociada a la distribución i de tipo 6 (SameWeeks)
dicc_PD6i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "SameWeeks":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD6i[int(row["DistributionId"])] = dp
        
#PD7,i=penalización asociada a la distribución i de tipo 7(Precedence) 
dicc_PD7i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "Precedence":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD7i[int(row["DistributionId"])] = dp
        
#PD8,i=penalización asociada a la distribución i de tipo 8(NotOverlap) 
dicc_PD8i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "NotOverlap":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD8i[int(row["DistributionId"])] = dp